In [22]:
import pandas as pd
import numpy as np 
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import HTML

In [23]:
!ls

Collaboration_trends.ipynb            data-import-NCBI.ipynb
Data_Analysis_and_Visualization.ipynb data-import-visualization.ipynb
Data_Processing.ipynb                 test2_file.txt
data-import-NCBI-entrez.ipynb


In [24]:
!cat test2_file.txt | head -10

Id	ArticleIds	AuthorList	DOI	EPubDate	FullJournalName	HasAbstract	LastAuthor	NlmUniqueID	PubDate	PubTypeList	RecordStatus	Source	Title
31734938	DictElement({'pubmed': ['31734938'], 'medline': [], 'doi': '10.1007/s13555-019-00342-1', 'pii': '10.1007/s13555-019-00342-1', 'rid': '31734938', 'eid': '31734938'}, attributes={})	['Ly K', 'Chang AY', 'Kiprono SK', 'Jose M', 'Smith MP', 'Beck K', 'Sekhon S', 'Muraguri I', 'Mungai M', 'Coates SJ', 'Thibodeaux Q', 'Hulse S', 'Gualberto M', 'Jeon C', 'Nakamura M', 'Bhutani T', 'Maurer T', 'Liao W']	10.1007/s13555-019-00342-1	2019 Nov 16	Dermatology and therapy	1	Liao W	101590450	2019 Nov 16	['Journal Article']	PubMed - as supplied by publisher	Dermatol Ther (Heidelb)	Implementation of an Ultraviolet Phototherapy Service at a National Referral Hospital in Western Kenya: Reflections on Challenges and Lessons Learned.
31734704	DictElement({'pubmed': ['31734704'], 'medline': [], 'pii': '5627697', 'doi': '10.1093/cid/ciz1104', 'rid': '31734704', 'eid':

In [25]:
# import a tab separated file
papers = pd.read_csv("test2_file.txt", sep = "\t")

In [26]:
#profile = ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}})

In [27]:
#profile.to_widgets()

In [28]:
# See all the columns in the dataframe 
papers.columns

Index(['Id', 'ArticleIds', 'AuthorList', 'DOI', 'EPubDate', 'FullJournalName',
       'HasAbstract', 'LastAuthor', 'NlmUniqueID', 'PubDate', 'PubTypeList',
       'RecordStatus', 'Source', 'Title'],
      dtype='object')

In [29]:
# See the first 20 items in the column author list
papers.AuthorList[:20]

0     ['Ly K', 'Chang AY', 'Kiprono SK', 'Jose M', '...
1                             ['Agan BK', 'Marconi VC']
2     ['Smit M', 'Perez-Guzman PN', 'Mutai KK', 'Cas...
3     ["K'oreje KO", 'Okoth M', 'Van Langenhove H', ...
4     ['Kirungu JN', 'Magwanga RO', 'Pu L', 'Cai X',...
5     ['Pérez-Mazliah D', 'Ndungu FM', 'Aye R', 'Lan...
6     ['Baratedi WM', 'Tshiamo WB', 'Mogobe KD', 'Mc...
7     ['Mwanga G', 'Lockwood S', 'Mujibi DFN', 'Yona...
8     ['Vikeli E', 'Widdick DA', 'Batey SF', 'Heine ...
9     ['Jemutai-Kimosop S', 'Orata F', 'Shikuku VO',...
10    ['Toombs C', 'Lonner B', 'Fazal A', 'Boachie-A...
11     ['El-Niweiri MAA', 'Moritz RFA', 'Lattorff HMG']
12    ['Sserumaga JP', 'Ortega-Beltran A', 'Wagacha ...
13    ['Vinayak S', 'Froum SJ', 'Liu K', 'Lim AC', '...
14    ['Kiwuwa-Muyingo S', 'Abongomera G', 'Mambule ...
15          ['Ding H', 'Kumar KR', 'Boiyo R', 'Zhao T']
16    ['Nthiwa D', 'Alonso S', 'Odongo D', 'Kenya E'...
17    ['Mwenda V', 'Niyomwungere A', 'Oyugi E', 

In [30]:
# remove any weird characters in the column for author 
remove_quote = papers.AuthorList.str.replace("'", "")
remove_brac = remove_quote.str.replace("[", "")
remove_close = remove_brac.str.replace("]", "")

# convert it to a list for other steps 
author_collab = list(remove_close)

In [31]:
author_collab

['Ly K, Chang AY, Kiprono SK, Jose M, Smith MP, Beck K, Sekhon S, Muraguri I, Mungai M, Coates SJ, Thibodeaux Q, Hulse S, Gualberto M, Jeon C, Nakamura M, Bhutani T, Maurer T, Liao W',
 'Agan BK, Marconi VC',
 'Smit M, Perez-Guzman PN, Mutai KK, Cassidy R, Kibachio J, Kilonzo N, Hallett TB',
 '"Koreje KO", Okoth M, Van Langenhove H, Demeestere K',
 'Kirungu JN, Magwanga RO, Pu L, Cai X, Xu Y, Hou Y, Zhou Y, Cai Y, Hao F, Zhou Z, Wang K, Liu F',
 'Pérez-Mazliah D, Ndungu FM, Aye R, Langhorne J',
 'Baratedi WM, Tshiamo WB, Mogobe KD, McFarland DM',
 'Mwanga G, Lockwood S, Mujibi DFN, Yonah Z, Chagunda MGG',
 'Vikeli E, Widdick DA, Batey SF, Heine D, Holmes NA, Bibb MJ, Martins DJ, Pierce NE, Hutchings MI, Wilkinson B',
 'Jemutai-Kimosop S, Orata F, Shikuku VO, Okello VA, Getenga ZM',
 'Toombs C, Lonner B, Fazal A, Boachie-Adjei O, Bastrom T, Pellise F, Ramadan M, Koptan W, ElMiligui Y, Zhu F, Qiu Y, Shufflebarger H',
 'El-Niweiri MAA, Moritz RFA, Lattorff HMG',
 'Sserumaga JP, Ortega-Bel

In [49]:
# Initialize TFIdf model with min_df set to 1
vectorizer = TfidfVectorizer(min_df=1)

# fit and transform the data
model = vectorizer.fit_transform(remove_close)

# return a numpy matrix with spaces filled with zeros done by the .todense method
# change all our numbers to 2 dp
print(model.todense().round(2))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [52]:
# find the mean across each nested array and convert it to a list
weights = np.asarray(model.mean(axis=0)).ravel().tolist()

# Take the output of the Tfidf vectorizer and the weights and turn it into a dataframe
weights_df = pd.DataFrame({'term': vectorizer.get_feature_names(), 'weight': weights})

# take the weights_df and sort by weight in descending order and return the first 20 values
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
367,jm,0.013204
845,van,0.012447
459,li,0.012436
171,da,0.010677
179,de,0.010468
193,dm,0.010270
864,wang,0.010126
368,jn,0.010024
356,jałoszyński,0.010000
627,obonyo,0.009863


In [50]:
HTML(weights_df.sort_values(by='weight', ascending=False).to_html())

,term,weight
367,jm,0.013204
845,van,0.012447
459,li,0.012436
171,da,0.010677
179,de,0.010468
193,dm,0.010270
864,wang,0.010126
368,jn,0.010024
356,jałoszyński,0.010000
627,obonyo,0.009863


In [45]:
author_collab

['Ly K, Chang AY, Kiprono SK, Jose M, Smith MP, Beck K, Sekhon S, Muraguri I, Mungai M, Coates SJ, Thibodeaux Q, Hulse S, Gualberto M, Jeon C, Nakamura M, Bhutani T, Maurer T, Liao W',
 'Agan BK, Marconi VC',
 'Smit M, Perez-Guzman PN, Mutai KK, Cassidy R, Kibachio J, Kilonzo N, Hallett TB',
 '"Koreje KO", Okoth M, Van Langenhove H, Demeestere K',
 'Kirungu JN, Magwanga RO, Pu L, Cai X, Xu Y, Hou Y, Zhou Y, Cai Y, Hao F, Zhou Z, Wang K, Liu F',
 'Pérez-Mazliah D, Ndungu FM, Aye R, Langhorne J',
 'Baratedi WM, Tshiamo WB, Mogobe KD, McFarland DM',
 'Mwanga G, Lockwood S, Mujibi DFN, Yonah Z, Chagunda MGG',
 'Vikeli E, Widdick DA, Batey SF, Heine D, Holmes NA, Bibb MJ, Martins DJ, Pierce NE, Hutchings MI, Wilkinson B',
 'Jemutai-Kimosop S, Orata F, Shikuku VO, Okello VA, Getenga ZM',
 'Toombs C, Lonner B, Fazal A, Boachie-Adjei O, Bastrom T, Pellise F, Ramadan M, Koptan W, ElMiligui Y, Zhu F, Qiu Y, Shufflebarger H',
 'El-Niweiri MAA, Moritz RFA, Lattorff HMG',
 'Sserumaga JP, Ortega-Bel

Wagacha JM, Pereira JM, Smith JM, Baeten JM

VAN Damme K,VAN Noort S, Vandoolaeghe P, Vanhuyse F, van der Laan M

Obonyo M has appeared in twice in an author list look into the collaborations and insitutions

Copeland RS

Bukusi EA